## Repo statistics for trydotnet

In [1]:
#r "nuget:Octokit, 0.32.0"
#r "nuget:NodaTime, 2.4.6"
using Octokit;
using NodaTime;
using NodaTime.Extensions;
using XPlot.Plotly;

Attempting to install package Octokit, version 0.32.0............................................................done!

Successfully added reference to package Octokit, version 0.32.0

Attempting to install package NodaTime, version 2.4.6..................done!

Successfully added reference to package NodaTime, version 2.4.6

In [2]:
var today = SystemClock.Instance.InUtc().GetCurrentDate();
var startOfTheMonth = today.With(DateAdjusters.StartOfMonth);
var startOfTheYear = new LocalDate(today.Year, 1, 1).AtMidnight();

In [3]:
var options = new ApiOptions();
var gitHubClient = new GitHubClient(new ProductHeaderValue("notebook"));

In [4]:
var createdIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheMonth.ToDateTimeUnspecified(),
                Filter = IssueFilter.Created
            };
var closedIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheMonth.ToDateTimeUnspecified(),
                State = ItemStateFilter.Closed
            };
var thisYearIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheYear.ToDateTimeUnspecified()
            };

In [5]:
var createdThisMonth = await gitHubClient.Issue.GetAllForRepository("dotnet", "try", createdIssuesRequest);
var closedThisMonth = await gitHubClient.Issue.GetAllForRepository("dotnet", "try", closedIssuesRequest);
var thisYearIssues = await gitHubClient.Issue.GetAllForRepository("dotnet", "try", thisYearIssuesRequest);

In [6]:
var openSoFar = createdThisMonth.Where(i => i.State.StringValue == "open");
var openByMonth = openSoFar.GroupBy(i => new { i.CreatedAt.Year, i.CreatedAt.Month})
                .Select(g => new {Date = g.Key, Count = g.Count()});
var closedSoFar = thisYearIssues.Where(i => i.State.StringValue == "closed");
var closedByMonth = closedSoFar.GroupBy(i => new { i.ClosedAt.Value.Year, i.ClosedAt.Value.Month})
                .Select(g => new {Date = g.Key, Count = g.Count()});

In [7]:
//display(createdThisMonth.Select(i => new {i.CreatedAt, i.Title, State = i.State.StringValue,  i.Number}));
var lineChart = Chart.Line(createdThisMonth.GroupBy(i => new DateTime(i.CreatedAt.Year,i.CreatedAt.Month, i.CreatedAt.Day)).Select(g => new Tuple<DateTime,int>(g.Key, g.Count())));
lineChart.WithTitle("Issue Open By Day");
display(lineChart);

In [8]:
display(openSoFar.Select(i => new {i.CreatedAt, i.Title, State = i.State.StringValue,  i.Number}));

index,CreatedAt,Title,State,Number
0,2019-09-16 20:09:30Z,Build on preview9,open,449
1,2019-09-15 20:26:02Z,System.ArgumentException: An item with the same key has already been added. Key: .wasm,open,445
2,2019-09-10 21:09:41Z,Kernel Info reply doesn't reflect the kernel language and the editor coloring is not correct,open,437
3,2019-09-07 09:56:08Z,"Android on screen function is not working, back space can't be used to edit",open,429
4,2019-09-06 01:29:27Z,Creating kernel extensions that are not part of other nuget packages,open,428
5,2019-09-05 23:22:12Z,Publish interactive packages,open,427
6,2019-09-05 12:30:15Z,WIP create notebook examples,open,425
7,2019-09-03 16:30:59Z,"After running a %%html / %%markdown / %%javascript, ""running"" indicator is not reset",open,423
8,2019-08-29 16:08:30Z,Try the .NET Kernel,open,408
9,2019-08-22 20:09:04Z,Adding extensions to the .NET kernel using nuget packages,open,395


In [9]:
//display(openByMonth);
var lineChart = Chart.Line(openByMonth.Select(g => new Tuple<DateTime, int>(new DateTime(g.Date.Year, g.Date.Month, 1),g.Count)));
lineChart.WithTitle("Issue Open By Month");
display(lineChart);